In [ ]:
import os,sys
import numpy as np
import chart_studio as cs
import chart_studio.plotly as py
import plotly.graph_objects as go

In [ ]:
import mmcv
import torch
from mmcv.runner import init_dist
from mmcv.utils import Config, build_from_cfg, DictAction, get_git_hash

from mmseg import __version__
from mmseg.apis import set_random_seed, train_segmentor
from mmseg.datasets import build_dataset
from mmseg.models import build_segmentor
from mmseg.utils import collect_env, get_root_logger

import xcit.semantic_segmentation
from xcit.semantic_segmentation.backbone import xcit
from mmseg.models.builder import BACKBONES,SEGMENTORS
BACKBONES.register_module('XCiT',True,xcit.XCiT)

In [ ]:
print(BACKBONES)

In [ ]:
# LOAD MODEL CONFIG
from mmcv.utils import Config
cfg = Config.fromfile("xcit/semantic_segmentation/configs/xcit/sem_fpn/sem_fpn_xcit_medium_24_p8_80k_ublarcv.py")
cfg.model.backbone['in_chans'] = 1
print(cfg.model)

In [ ]:
DEVICE = torch.device('cuda:0')

In [ ]:
model = build_segmentor(
        cfg.model,
        train_cfg=cfg.get('train_cfg'),
        test_cfg=cfg.get('test_cfg')).to(DEVICE)

In [ ]:
print(model)

In [ ]:
import ROOT as rt
from larmatch_dataset import larmatchDataset

In [ ]:
batch_size=1
test = larmatchDataset( filefolder="./", random_access=True )
print("NENTRIES: ",len(test))
    
loader = torch.utils.data.DataLoader(test,batch_size=batch_size,collate_fn=larmatchDataset.collate_fn)

In [ ]:
# Get data
batch = next(iter(loader))
print(batch[0].keys())

In [ ]:
# check sparse image
for p in range(3):
    num_unique = np.unique( batch[0]['coord_%d'%(p)], axis=0 ).shape[0]
    print("plane ",p)
    print("  coord shape: ",batch[0]['coord_%d'%(p)].shape)
    print("  num unique: ",num_unique)

In [ ]:
# make a dense y-plane
imgy = np.zeros( (1024, 3456), dtype=np.float32)
coord = batch[0]['coord_2']
feat  = batch[0]['feat_2']
imgy[ coord[:,0], coord[:,1] ] = feat[:,0]
print(imgy.shape)

In [ ]:
# look at the image

plotimg = np.clip(imgy,0,255)

xaxis = np.linspace( 0, 3456, endpoint=False, num=3456 )
yaxis = np.linspace( 2400, 2400+1024*6, endpoint=False, num=1024 )

layout_yplane = go.Layout(
    title='Y-PLANE WIRE IMAGE',
    autosize=True,
    hovermode='closest',
    showlegend=False)
    
heatmap = {
    #"type":"heatmapgl",                                                                                                                                                                  
    "type":"heatmap",
    "z":plotimg,
    "x":xaxis,
    "y":yaxis,
    "colorscale":"Jet",
}

fig = go.Figure(data=[heatmap],layout=layout_yplane)
fig.show()

In [ ]:
"""
    parser.add_argument('--world_size', default=1, type=int,
                        help='number of distributed processes')
    parser.add_argument('--dist_url', default='env://', help='url used to set up distributed training')
    parser.add_argument('--test-freq', default=1, type=int, help='Number of epochs between \                                                                                                  
                                                                  validation runs.')

    parser.add_argument('--full_crop', action='store_true', help='use crop_ratio=1.0 instead of the\                                                                                          
                                                                  default 0.875 (Used by CaiT).')
    parser.add_argument("--pretrained", default=None, type=str, help='Path to pre-trained checkpoint')
    parser.add_argument('--surgery', default=None, type=str, help='Path to checkpoint to copy the \                                                                                           
                                                                   patch projection from. \                                                                                                   
                                                                   Can improve stability for very \                                                                                           
                                                                   large models.')

"""
# os.environ["RANK"] = '0'
# os.environ['WORLD_SIZE'] = '1'
# os.environ['LOCAL_RANK'] = '0'
# os.environ['MASTER_ADDR'] = '128.0.0.0'
# os.environ['MASTER_PORT'] = '8001'

# class HackArgs:
#     def __init__(self):
#         self.world_size = 1
#         self.dist_url = 'env://'
#         self.test_freq = 1
#         self.full_crop = False
#         self.pretrained = None
#         self.surgery = None
        
# args = HackArgs()
        
# util.init_distributed_mode(args)

In [ ]:
print("min: ",np.min(imgy))
print("max: ",np.max(imgy))

In [ ]:
img_t = torch.from_numpy(imgy.reshape((1,1,1024,3456))).to(DEVICE)
img_t.shape

In [ ]:
with torch.no_grad():
    out = model.forward_dummy(img_t)
    for n,subout in enumerate(out):
        print("[%d] "%(n),subout.shape)

In [ ]:
#cfg
type(REGISTRY)
type(cfg)
print(REGISTRY)